In [ ]:
##1. OLS is a bad option to estimate the model parameters becuase you need more observations than variables. 
## For that same reason we can use process of elimination to rule out ridge regression as it would result in high variance.
## Thus, we would use lasso selection.

##2. (f)

##3. (f)

##4. I would say the model suffers from high bias as it is underfitting, and we should reduce the regularization parameter λ.

In [4]:
##5a.
import boto3
import pandas as pd; pd.set_option('display.max_columns', 100)
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso, LassoCV

s3 = boto3.resource('s3')
bucket_name = 'omar-vargas-bucket'
bucket = s3.Bucket(bucket_name)

file_key = 'CarPrice_Assignment.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

#Reading the data-file
car_price = pd.read_csv(file_content_stream)
car_price.head()

,car_ID,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,carlength,carwidth,carheight,curbweight,enginetype,cylindernumber,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,1,3,alfa-romero giulia,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,48.8,2548,dohc,four,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0
1,2,3,alfa-romero stelvio,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,48.8,2548,dohc,four,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0
2,3,1,alfa-romero Quadrifoglio,gas,std,two,hatchback,rwd,front,94.5,171.2,65.5,52.4,2823,ohcv,six,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0
3,4,2,audi 100 ls,gas,std,four,sedan,fwd,front,99.8,176.6,66.2,54.3,2337,ohc,four,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0
4,5,2,audi 100ls,gas,std,four,sedan,4wd,front,99.4,176.6,66.4,54.3,2824,ohc,five,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0


In [5]:
##5b.
def l2_normalization(X):
    
    x_mean = X.mean()
    l2 = np.sqrt(sum(X**2))
    return (X - x_mean) / l2

# defining and storing the coef
coef = list()

# Creating the for loop for 1,000 iteractions
for i in range(0, 1000):
    
    # defining the input and target variables
    X = car_price[['wheelbase', 'enginesize', 'horsepower', 'compressionratio', 'peakrpm', 'citympg', 'highwaympg']]
    Y = car_price['price']

    # splitting the data into train and test
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2)
    
    X_train = X_train.apply(l2_normalization, axis = 1)
    X_test = X_test.apply(l2_normalization, axis = 1)
    
    # running the lasso to see cross validation and best lambda
    lasso_cv = LassoCV(cv = 5, max_iter = 10000).fit(X_train, Y_train)

    # extracting the best lamba
    lasso_md = Lasso(alpha = lasso_cv.alpha_, max_iter = 10000).fit(X_train, Y_train)
    
    # storing the coef
    coef.append(lasso_md.coef_)
    
# making the list a data frame
coef_data = pd.DataFrame(coef)
coef_data.head()

,0,1,2,3,4,5,6
0,-477846.360672,584926.392078,387817.109032,1.663142e+06,-0.0,-869331.485688,-484345.481720
1,-37067.199992,623145.016262,320728.669374,1.671067e+06,-0.0,-550709.343306,-906245.705154
2,-50959.493265,798609.007228,618345.667974,1.884138e+06,-0.0,-0.000000,-771458.167592
3,-133319.910761,860954.058064,609801.361949,1.946398e+06,-0.0,-204849.681896,-427519.652549
4,-75118.938697,889246.886361,682647.867533,2.158363e+06,-0.0,-593599.082000,-0.000000


In [6]:
def sum_zeros(X):
    
    return sum(X == 0)

coef_data= coef_data.apply(sum_zeros, axis = 1)
coef_data.head()

0    1
1    1
2    2
3    1
4    2
dtype: int64

In [7]:
coef_data= pd.DataFrame(coef)
sum(coef_data[4]== 0.0)

971

In [ ]:
## The results show that peakrpm had a estimated coefficient of 0 over 500 times, so it should be excluded from the new model

In [8]:
# dropping peakrpm variable
X_train= X_train.drop(columns= ['peakrpm'], axis= 1)
X_test= X_test.drop(columns= ['peakrpm'], axis= 1)

In [9]:
##5c.
def l2_normalization(X):
    
    x_mean= np.mean(X)
    l2 = np.sqrt(sum(X**2))
    return (X - x_mean) / l2


X_train = X_train.apply(l2_normalization, axis = 1)
X_test = X_test.apply(l2_normalization, axis = 1)

In [10]:
##5d.
from sklearn.linear_model import LinearRegression

# building the linear regression 
lm_md = LinearRegression().fit(X_train, Y_train)

# predicting on test data
lm_pred = lm_md.predict(X_test)

# computing the mse 
mse1 = np.mean(np.power(Y_test - lm_pred, 2))
print('The mse of the linear model is:', mse1)

The mse of the linear model is: 14001200.250868835


In [11]:
##5e. 
# building the ridge regression
for i in range(0,100):
    
    # ridge regression
    ridge_cv = RidgeCV(alphas = [0.001, 0.01, 0.1, 1, 10, 100], cv =5).fit(X_train, Y_train)

    # extracting the best lamba
    cv_lambda= ridge_cv.alpha_

    # building the ridge model
    ridge_md= Ridge(alpha = cv_lambda).fit(X_train, Y_train)

    # predicting on test
    ridge_pred = ridge_md.predict(X_test)

# computing the mse 
mse2 = np.mean(np.power(Y_test - ridge_pred, 2))
print('The mse of the ridge model is:', mse2)

The mse of the ridge model is: 15087661.766270228


In [ ]:
##5f. From the results, we see that the mse for the linear model is smaller, so we should that model to predict car prices.